# Creating classifier for tiles
* Based off: Deep Learning for Identifying Metastatic Breast Cancer arXiv:1606.05718v1

Differences:
* Use inception v3 (not google lenet)
* how many samples did they generate?

Transfer Learning code from:
http://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

In [1]:
import os
import sys
import glob
import random
import pickle
import numpy as np
from PIL import Image
import time
import copy

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from IPython.display import display, HTML
from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.transforms as transforms
import torch.utils.data
import torchvision.models as models
from torchvision import datasets, models, transforms
import torch.optim as optim
from torch.optim import lr_scheduler


%reload_ext autoreload
%autoreload 2
%matplotlib inline

%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0
print(torch.cuda.current_device())

use_gpu = torch.cuda.is_available()

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0
0


In [2]:
# Base Directory where data is stored
data_dir = '/media/rene/Data/camelyon_out/tiles_299_1t'

batch_size = 16
num_workers = 1

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'valid']}

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size,
                                             shuffle=True, num_workers=num_workers)
              for x in ['train', 'valid']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'valid']}
class_names = image_datasets['train'].classes

## Inception model is weird
* Input may be normalized differently than other imagenet models
* The output is a tuple consisting of: (actual ouput, aux loss)
* To train full model must sum this aux loss
* Making prediction is different if in training phase or not. In training must look at 1st variable in tuple, in validation you just have normal output.

Info:
https://github.com/ahirner/pytorch-retraining/blob/master/retrain_benchmark_bees.ipynb
https://discuss.pytorch.org/t/imagenet-example-with-inception-v3/1691/22

In [3]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'valid']:
            if phase == 'train':
                scheduler.step()
                model.train(True)  # Set model to training mode
            else:
                model.train(False)  # Set model to evaluate mode
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for data in dataloaders[phase]:
                # get the inputs
                inputs, labels = data

                # wrap them in Variable
                if use_gpu:
                    inputs = Variable(inputs.cuda())
                    labels = Variable(labels.cuda())
                else:
                    inputs, labels = Variable(inputs), Variable(labels)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                outputs = model(inputs)
                
                # for nets that have multiple outputs such as inception
                if isinstance(outputs, tuple):
                    loss = sum((criterion(o,labels) for o in outputs))
                else:
                    loss = criterion(outputs, labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    _, preds = torch.max(outputs[0].data, 1)
                    loss.backward()
                    optimizer.step()
                else:
                    _, preds = torch.max(outputs.data, 1)

                # statistics
                running_loss += loss.data[0] * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best valid Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [4]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    for i, data in enumerate(dataloaders['valid']):
        inputs, labels = data
        if use_gpu:
            inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
        else:
            inputs, labels = Variable(inputs), Variable(labels)

        outputs = model(inputs)
        _, preds = torch.max(outputs.data, 1)

        for j in range(inputs.size()[0]):
            images_so_far += 1
            ax = plt.subplot(num_images//2, 2, images_so_far)
            ax.axis('off')
            ax.set_title('predicted: {}'.format(class_names[preds[j]]))
            imshow(inputs.cpu().data[j])

            if images_so_far == num_images:
                model.train(mode=was_training)
                return
    model.train(mode=was_training)

In [5]:
model_ft = models.inception_v3(pretrained=True)

# for p in model_ft.parameters():
#     p.requires_grad=False

num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 2)

if use_gpu:
    model_ft = model_ft.cuda()

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [6]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=25)

Epoch 0/24
----------
train Loss: 1.8403 Acc: 0.7495
valid Loss: 0.1999 Acc: 0.9286
Epoch 1/24
----------
train Loss: 0.5357 Acc: 0.8772
valid Loss: 0.2768 Acc: 0.8750
Epoch 2/24
----------
train Loss: 0.4050 Acc: 0.9159
valid Loss: 0.2036 Acc: 0.8929
Epoch 3/24
----------
train Loss: 0.3901 Acc: 0.9226
valid Loss: 0.1108 Acc: 0.9286
Epoch 4/24
----------
train Loss: 0.3204 Acc: 0.9323
valid Loss: 0.2298 Acc: 0.9107
Epoch 5/24
----------
train Loss: 0.2871 Acc: 0.9449
valid Loss: 0.1679 Acc: 0.9107
Epoch 6/24
----------
train Loss: 0.2267 Acc: 0.9545
valid Loss: 0.2981 Acc: 0.8750
Epoch 7/24
----------
train Loss: 0.1765 Acc: 0.9691
valid Loss: 0.2279 Acc: 0.9107
Epoch 8/24
----------
train Loss: 0.2006 Acc: 0.9584
valid Loss: 0.2177 Acc: 0.9107
Epoch 9/24
----------
train Loss: 0.1309 Acc: 0.9778
valid Loss: 0.2577 Acc: 0.9107
Epoch 10/24
----------
train Loss: 0.1305 Acc: 0.9739
valid Loss: 0.2019 Acc: 0.9107
Epoch 11/24
----------
train Loss: 0.1185 Acc: 0.9797
valid Loss: 0.3141 Ac